In [1]:
import os, time, glob, socket
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import csv
import pickle
from sklearn.preprocessing import StandardScaler
random.seed(123)
pd.set_option("display.max_columns", None)
from sklearn.preprocessing import LabelEncoder
os.getcwd()

'C:\\Users\\user\\Desktop\\FINTECH'

In [2]:
os.chdir(r'D:\fintech')

# 原始的所有變數

In [3]:
X_train = pd.read_csv("X_train.csv")
X_test = pd.read_csv("X_test.csv")
y_train = pd.read_csv("Y_train.csv")

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

# 上下合併  X_train X_test 
X_all=pd.concat([X_train,X_test],axis=0,ignore_index=True,verify_integrity=False)

(2132800, 150)
(533200, 150)
(2132800, 2)


# 連續的

In [4]:
num=['acc_now_delinq','acc_open_past_24mths','all_util','annual_inc','annual_inc_joint','avg_cur_bal',
'bc_open_to_buy','bc_util','chargeoff_within_12_mths','collection_recovery_fee','collections_12_mths_ex_med',
'delinq_2yrs','deferral_term','delinq_amnt','dti','dti_joint', 'fico_range_high','fico_range_low','funded_amnt',
'funded_amnt_inv','hardship_amount','hardship_dpd','hardship_last_payment_amount','hardship_length',
'hardship_payoff_balance_amount','il_util' ,'inq_fi','inq_last_12m','inq_last_6mths','installment','last_fico_range_high',
'last_fico_range_low','last_pymnt_amnt','loan_amnt','max_bal_bc', 'mo_sin_old_il_acct','mo_sin_old_rev_tl_op',
'mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mort_acc','mths_since_last_delinq','mths_since_last_major_derog',
'mths_since_last_record','mths_since_rcnt_il','mths_since_recent_bc','mths_since_recent_bc_dlq',
'mths_since_recent_inq','mths_since_recent_revol_delinq','num_accts_ever_120_pd','num_actv_bc_tl',
'num_actv_rev_tl','num_bc_sats','num_bc_tl','num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0',
'num_sats','num_tl_120dpd_2m','num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m','open_acc','open_acc_6m',
'open_act_il','open_il_12m','open_il_24m','open_rv_12m','open_rv_24m','orig_projected_additional_accrued_interest',
'out_prncp','out_prncp_inv','pct_tl_nvr_dlq','percent_bc_gt_75','policy_code','pub_rec','pub_rec_bankruptcies','recoveries',
'revol_bal','revol_bal_joint','sec_app_chargeoff_within_12_mths','sec_app_collections_12_mths_ex_med','sec_app_fico_range_high',
'sec_app_fico_range_low','sec_app_inq_last_6mths','sec_app_mort_acc','sec_app_mths_since_last_major_derog',
'sec_app_num_rev_accts','sec_app_open_acc','sec_app_open_act_il','sec_app_revol_util','settlement_amount',
'settlement_percentage','settlement_term' ,'tax_liens','tot_coll_amt','tot_cur_bal','tot_hi_cred_lim','total_acc'
,'total_bal_ex_mort','total_bal_il','total_bc_limit','total_cu_tl','total_il_high_credit_limit','total_pymnt','total_pymnt_inv'
,'total_rec_int','total_rec_late_fee','total_rec_prncp','total_rev_hi_lim' ]

In [5]:
X_num=X_all[num]
print(X_num.shape)

(2666000, 110)


# 有文字的---drop url

In [6]:
char=['addr_state','application_type','debt_settlement_flag','debt_settlement_flag_date','desc','earliest_cr_line','emp_length','emp_title','grade',
 'hardship_end_date','hardship_flag','hardship_loan_status','hardship_reason','hardship_start_date','hardship_status',
 'hardship_type','home_ownership','initial_list_status','int_rate','issue_d','last_credit_pull_d','last_pymnt_d',
 'next_pymnt_d','payment_plan_start_date','purpose','pymnt_plan','revol_util','sec_app_earliest_cr_line','sub_grade',
'settlement_date','settlement_status','term','title','verification_status','verification_status_joint','zip_code']
print(len(char))
X_char=X_all[char]
print(X_char.shape)

36
(2666000, 36)


# 沒有意義

In [7]:
NO=['Unnamed: 0','Unnamed: 0.1', 'id', 'member_id']
print(len(NO))
X_id=X_all['id']
print(X_id.shape)

4
(2666000,)


# missing vale

## num

In [8]:
# 看 MISSING VALUE 
num_isnull=X_num.isnull().sum(axis=0)

# 沒有 MISSING VALUE 的部分(變數)
num_without_na=num_isnull[num_isnull == 0]
X_num_without_na=X_num[num_without_na.index]

# 有 MISSING VALUE 的部分(變數)
num_with_na=num_isnull[num_isnull != 0]
X_num_with_na=X_num[num_with_na.index]

## 刪除 有 >20% missing value 的變數 ---
num_with_na_le20=num_with_na[num_with_na < 0.2*X_num.shape[0]]
X_with_na_le20=X_num[num_with_na_le20.index]

print(X_num_without_na.shape)
print(X_with_na_le20.shape)

(2666000, 0)
(2666000, 67)


# num 補值

In [37]:
# 需要補值的整筆 data
use_var=['id']+num_without_na.index.tolist()+num_with_na_le20.index.tolist() #加入 id
X_num_use_var=X_all[use_var]
print(X_num_use_var.shape)

# data中沒有 missing value 的部分(觀測值)
filter=X_num_use_var.isnull().sum(axis=1)==0
X_num_use_var_rowwithoutna=X_num_use_var[filter] # 含有 NA的 觀測值分
print(X_num_use_var_rowwithoutna.shape)

# data中有 missing value 的部分(觀測值)
filter=X_num_use_var.isnull().sum(axis=1)!=0
X_num_use_var_rowwithna=X_num_use_var[filter] # 含有 NA的 觀測值分
print(X_num_use_var_rowwithna.shape)

(2666000, 68)
(2193986, 68)
(472014, 68)


In [38]:
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


In [39]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor

In [40]:
from sklearn.impute import KNNImputer

In [41]:
# 用 mean 補值
var=X_num_use_var.columns.tolist()

for i in range(0,len(var)): 
     X_num_use_var[str(var[i])]=X_num_use_var[str(var[i])].fillna(X_num_use_var[str(var[i])].mean())

C:\Users\user\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [44]:
X_num_use_var.head()

,id,acc_now_delinq,acc_open_past_24mths,annual_inc,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,collection_recovery_fee,collections_12_mths_ex_med,delinq_2yrs,delinq_amnt,dti,fico_range_high,fico_range_low,funded_amnt,funded_amnt_inv,inq_last_6mths,installment,last_fico_range_high,last_fico_range_low,last_pymnt_amnt,loan_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,open_acc,out_prncp,out_prncp_inv,pct_tl_nvr_dlq,percent_bc_gt_75,policy_code,pub_rec,pub_rec_bankruptcies,recoveries,revol_bal,tax_liens,tot_coll_amt,tot_cur_bal,tot_hi_cred_lim,total_acc,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,total_pymnt,total_pymnt_inv,total_rec_int,total_rec_late_fee,total_rec_prncp,total_rev_hi_lim
0,2115583,0.0,3.0,58000.0,20068.0,448.0,95.6,0.0,0.0,0.0,0.0,0.0,24.65,729.0,725.0,3000.0,3000.0,0.0,93.86,694.0,690.0,2257.65,3000.0,113.0,158.0,18.0,17.0,1.0,50.0,17.0,0.0,4.0,6.0,4.0,8.0,4.0,9.0,16.0,6.0,12.0,0.0,0.0,0.0,0.0,12.0,0.00,0.00,100.0,100.0,1.0,0.0,0.0,0.0,14362.0,0.0,0.0,240820.0,283149.0,21.0,43597.0,10100.0,41899.0,3194.930000,3194.93,194.93,0.0,3000.00,28800.0
1,325608,0.0,5.0,80000.0,41635.0,1943.0,22.3,0.0,0.0,0.0,0.0,0.0,17.51,749.0,745.0,24000.0,24000.0,0.0,761.08,674.0,670.0,761.08,24000.0,116.0,102.0,6.0,5.0,3.0,6.0,5.0,0.0,1.0,1.0,2.0,2.0,10.0,2.0,6.0,1.0,5.0,0.0,0.0,0.0,5.0,5.0,16690.99,16690.99,100.0,0.0,1.0,0.0,0.0,0.0,557.0,0.0,0.0,208174.0,216203.0,19.0,67013.0,2500.0,71703.0,9109.470000,9109.47,1800.46,0.0,7309.01,2500.0
2,334066,0.0,3.0,45000.0,12320.0,2926.0,39.0,0.0,0.0,0.0,0.0,0.0,23.33,724.0,720.0,10000.0,10000.0,0.0,324.23,759.0,755.0,324.23,10000.0,158.0,146.0,15.0,12.0,1.0,15.0,12.0,0.0,4.0,5.0,4.0,4.0,12.0,7.0,7.0,5.0,11.0,0.0,0.0,0.0,1.0,11.0,7002.98,7002.98,90.5,0.0,1.0,0.0,0.0,0.0,2531.0,0.0,64.0,135519.0,150310.0,21.0,32492.0,4800.0,35450.0,3885.020000,3885.02,888.00,0.0,2997.02,7700.0
3,775123,0.0,4.0,76000.0,22974.0,60001.0,9.2,0.0,0.0,0.0,0.0,0.0,28.47,784.0,780.0,5000.0,5000.0,1.0,153.64,774.0,770.0,4389.73,5000.0,115.0,154.0,5.0,5.0,2.0,5.0,5.0,0.0,7.0,8.0,7.0,9.0,7.0,8.0,10.0,8.0,13.0,0.0,0.0,0.0,3.0,13.0,0.00,0.00,100.0,0.0,1.0,0.0,0.0,0.0,6136.0,0.0,0.0,298668.0,367844.0,19.0,139078.0,66100.0,131360.0,5133.402472,5133.40,133.40,0.0,5000.00,67600.0
4,1227286,0.0,4.0,72000.0,1285.0,7714.0,61.0,0.0,0.0,0.0,0.0,0.0,10.93,669.0,665.0,12000.0,12000.0,1.0,416.28,669.0,665.0,12268.23,12000.0,72.0,88.0,2.0,2.0,0.0,27.0,6.0,0.0,8.0,11.0,11.0,13.0,4.0,14.0,17.0,11.0,16.0,0.0,0.0,0.0,2.0,16.0,0.00,0.00,100.0,66.7,1.0,0.0,0.0,0.0,13406.0,0.0,0.0,17986.0,32567.0,21.0,17986.0,19800.0,8667.0,12248.163340,12248.16,248.16,0.0,12000.00,23900.0


In [46]:
# 分 train test
filter=X_num_use_var.index<X_train.shape[0]
X_num_imp_train=X_num_use_var[filter]
filter=X_num_use_var.index>=(X_all.shape[0]-X_test.shape[0])
X_num_imp_test=X_num_use_var[filter]

# 匯出 csv
X_num_imp_train.to_csv(r"X_num_imp_train.csv")
X_num_imp_test.to_csv(r"X_num_imp_test.csv")

# CHAR 處理

In [47]:
# 看 MISSING VALUE 
char_isnull=X_char.isnull().sum(axis=0)

# 沒有 MISSING VALUE 的部分(變數)
char_without_na=char_isnull[char_isnull == 0]
X_char_without_na=X_char[char_without_na.index]

# 有 MISSING VALUE 的部分(變數)
char_with_na=char_isnull[char_isnull != 0]
X_char_with_na=X_char[char_with_na.index]

# 刪除 擁有 Missing value >20% 的變數
char_with_na_le20=char_with_na[char_with_na < 0.2*X_char.shape[0]]
X_char_with_na_le20=X_char[char_with_na_le20.index]

print(X_char_without_na.shape)
print(X_char_with_na_le20.shape)

(2666000, 0)
(2666000, 22)


In [48]:
# char的整筆變數
use_var=['id']+char_without_na.index.tolist()+char_with_na_le20.index.tolist() #加入 id
X_char_use_var=X_all[use_var]
print(X_char_use_var.shape)

# 先不補植---直接分 train test
filter=X_char_use_var.index<X_train.shape[0]
X_char_train=X_char_use_var[filter]
filter=X_char_use_var.index>=(X_all.shape[0]-X_test.shape[0])
X_char_test=X_char_use_var[filter]

# 匯出 csv
X_char_train.to_csv(r"X_char_train.csv")
X_char_test.to_csv(r"X_char_test.csv")

(2666000, 23)


## 檔案讀進來--合併就可以跑模型了

In [49]:
X_num_imp_train = pd.read_csv("X_num_imp_train.csv",index_col=0)
X_char_train = pd.read_csv("X_char_train.csv",index_col=0)
X_num_imp_test = pd.read_csv("X_num_imp_test.csv",index_col=0)
X_char_test = pd.read_csv("X_char_test.csv",index_col=0)

C:\Users\user\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


## train_num & test_num 一起做 PCA

In [50]:
from sklearn.decomposition import PCA 
X_num_imp_train_id=X_num_imp_train['id']
X_num_imp_test_id=X_num_imp_test['id']
# 拿掉 id
X_num_imp_train=X_num_imp_train.drop(columns=['id'])
X_num_imp_test=X_num_imp_test.drop(columns=['id'])

# 标准化数据
ss = StandardScaler()
X_num_imp_train_std= ss.fit_transform(X_num_imp_train)
X_num_imp_test_std=ss.fit_transform(X_num_imp_test)

###  PCA
pca=PCA(n_components=0.8,copy=True, whiten=False)
X_num_imp_train_pca=pca.fit_transform(X_num_imp_train_std)

#print(pca.explained_variance_ratio_) 
#print(pca.explained_variance_)
print(pca.n_components_)

# 查看降维后的数据
X_num_imp_train_pca = pd.DataFrame(data=X_num_imp_train_pca)
X_num_imp_train_pca.head()

# 加回 id
X_num_imp_train_pca['id']=X_num_imp_train_id

21


## 把原始 testing 映射到主成分组成的子空间中

In [51]:
X_num_imp_test_pca=pd.DataFrame(data=pca.transform(X_num_imp_test_std)) # 標準化之後的
X_num_imp_test_pca['id']=X_num_imp_test_id.reset_index().id  # 加回 id

In [52]:
# 把 pca 後的 加上 char的
X_train_new=pd.merge(X_num_imp_train_pca,X_char_train)
X_test_new=pd.merge(X_num_imp_test_pca,X_char_test)
print(X_train_new.shape)
print(X_test_new.shape)

(2132800, 44)
(533200, 44)
